In [1]:
import time
import matplotlib
import io
matplotlib.use("Agg")
import matplotlib.pyplot as plt
start_time = time.time()


import numpy as np
from skimage.io import imread
from skimage.filters import gaussian, threshold_local
from skimage.measure import find_contours, moments
from skimage.draw import line
from skimage.color import gray2rgb
import math

from PIL import Image


In [2]:
image = imread('./DLMIA.png', as_gray=True)

Once the image is loaded lets delete some of the noise and threshold the image

In [3]:
image_gauss = gaussian(image, 70, multichannel=False) # Makes the image smoother
block_size = 9
local_thresh = threshold_local(image, block_size)
binary_local = image_gauss > local_thresh

Let's find the contours, and define a funtion to find the areas of those contours.

In [4]:
contours = find_contours(binary_local, 0.9)

# Taken from https://groups.google.com/forum/#!topic/scikit-image/--pXW-fPbGg
def polygonArea(poly):
    """
    Return area of an unclosed polygon.
    
    :see: https://stackoverflow.com/a/451482
    :param poly: (n,2)-array
    """
    # we need a plain list for the following operations
    if isinstance(poly, np.ndarray):
        poly = poly.tolist()
    segments = zip(poly, poly[1:] + [poly[0]])
    return 0.5 * abs(sum(x0*y1 - x1*y0
                         for ((x0, y0), (x1, y1)) in segments))
areas = map(polygonArea, contours)


Get the larges area, and create a black background

In [5]:
np_areas = np.array(list(areas))
result = np.where(np_areas == np.amax(np_areas))
max_index = result[0].item()
black = binary_local > 256

Plots the coordinates in top of the black image

In [6]:
fig, ax = plt.subplots()

ax.imshow(black, cmap=plt.cm.gray)
contour = contours[max_index]

ax.plot(contour[:, 1], contour[:, 0])
plt.axis('off')
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.show()

/home/richard/.pyenv/versions/3.7.3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


Saves the plot into a buffer

In [7]:

buf = io.BytesIO()
fig.savefig(buf,
    format='jpeg',
    bbox_inches='tight',
    pad_inches=0)
buf.seek(0)
im = Image.open(buf)

img_only_borders = np.array(im)
buf.close()

In [8]:
# plt.imshow(img)

from skimage.feature import corner_harris, corner_peaks

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
img_only_borders_grayscale = rgb2gray(img_only_borders)

coords = corner_peaks(corner_harris(img_only_borders_grayscale), min_distance=2)
# fig, ax = plt.subplots()
# ax.imshow(img, cmap=plt.cm.gray)
# ax.plot(coords[:, 1], coords[:, 0], '+r', markersize=15)
# plt.show()

In [9]:
def get_center_point(img):
    img_momentum = moments(img)
    center_x = img_momentum[1, 0] / img_momentum[0, 0]
    center_y = img_momentum[0, 1] / img_momentum[0, 0]
    
    return (center_x, center_y)

def find_corner(img, coords):
    center_x , center_y = get_center_point(img)
    
    corners = {'up_left': (0,0), 'down_left': (0,0), 'up_right':(0,0), 'down_right': (0,0)}
    distance = {'up_left': 0, 'down_left': 0, 'up_right':0, 'down_right': 0}
    
    for coord in coords:
        [y, x] = coord
        dist = math.sqrt((x - center_x)**2 + (y - center_y)**2)
        
        if x < center_x and y < center_y and distance['up_left'] < dist:
            corners['up_left'] = (x, y)
            distance['up_left'] = dist
            
        elif x > center_x and y < center_y and distance['up_right'] < dist:
            corners['up_right'] = (x, y)
            distance['up_right'] = dist
            
        elif x > center_x and y > center_y and distance['down_right'] < dist:
            corners['down_right'] = (x, y)
            distance['down_right'] = dist
            
        elif x < center_x and y > center_y and distance['down_left'] < dist:
            corners['down_left'] = (x, y)
            distance['down_left'] = dist
    
    return corners
            
    

corners = find_corner(img_only_borders_grayscale, coords)
fig, ax = plt.subplots()

ax.imshow(img_only_borders_grayscale, cmap=plt.cm.gray)
ax.plot([corners['up_right'][0]], [corners['up_right'][1]], '+r', markersize=15)
ax.plot([corners['up_left'][0]], [corners['up_left'][1]], '+r', markersize=15)
ax.plot([corners['down_right'][0]], [corners['down_right'][1]], '+r', markersize=15)
ax.plot([corners['down_left'][0]], [corners['down_left'][1]], '+r', markersize=15)

plt.show()

In [10]:
fig.savefig("local.png")
print(corners)
print("--- %s seconds ---" % (time.time() - start_time))


{'up_left': (48, 55), 'down_left': (23, 337), 'up_right': (322, 45), 'down_right': (341, 341)}
--- 8.17365837097168 seconds ---


In [11]:
img_only_borders.shape

(369, 366, 3)